# U-Net MODEL
A network architecture called "U-Net". The name of this network architecture comes from it's U-like shape when shown in a diagram like this (image from [U-net entry on wikipedia](https://en.wikipedia.org/wiki/U-Net)): 

<img src="images/U-net_example_wikipedia.png" alt="U-net Image" width="600"/>

U-nets are commonly used for image segmentation. This architecture features a series of down-convolutions connected by max-pooling operations, followed by a series of up-convolutions connected by upsampling and concatenation operations. Each of the down-convolutions is also connected directly to the concatenation operations in the upsampling portion of the network.

In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install keras.engine

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras.engine (from versions: none)
ERROR: No matching distribution found for keras.engine

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate

# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## 1. The "Depth" of U-Net
the depth of U-Net is equal to the number of down-convolutions. In the image above, the depth is 4 because there are 4 down-convolutions running down the left side including the very bottom of the U.
Here we'll use a U-Net depth of 2, meaning we'll have 2 down-convolutions in our network.

### 1.1 Input layers and its "depth"

Here we will be doing 3D image segmentation whcih has "length" in addition to "height" and "width".

The shape of imput layer is (num_channels, height, widht, length), where num_channles is like color channels in an image, height, widht and length are just the size of the inpput

here,
num_channels = 4, 
height = 160,
width = 160,
length = 160

In [10]:
# defining an input layer tensor of the shape we'll use

input_layer = Input(shape=(4,160,160,16))
input_layer

<KerasTensor shape=(None, 4, 160, 160, 16), dtype=float32, sparse=None, name=keras_tensor>

Keras shape has "None" or "?" as the very first dimension. this will bethe batch size. so the dimensions of the tensor are: (batch_size, num_channles, height, width, length)

## 2. Contracting (downward) path

we'll start by constructing  the downward path in our network (the left side). the (height, width, length) if the input gets smaller as we move down this path, and the number of channels increases

## 2. Contracting (downward) Path 
Here you'll start by constructing the downward path in your network (the left side of the U-Net).  The `(height, width, length)` of the input gets smaller as you move down this path, and the number of channels increases.

### 2.1 Depth 0

By "depth 0" here, we're referring to the depth of the first down-convolution in the U-net.

The number of filters is specified for each depth and for each layer( layer 1, 2 etc) within that depth.

The formula to use for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i})$$

Where $i$ is the current depth.

So at depth $i=0$:
$$filters_{0} = 32 \times (2^{0}) = 32$$

### 2.2 Layer 0
There are two convolutional layers for each depth

In [11]:
# defining a Conv3D tensor with 32 filters

down_depth_0_layer_0 = Conv3D(filters=32,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                              )(input_layer)
# constructing this on input layer 
down_depth_0_layer_0

<KerasTensor shape=(None, 32, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_1>

notice that with 32 filters, the result we got abpve is a tensor with 32 channels

In [12]:
# add a relu activation to layer 0 of depth 0
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0

<KerasTensor shape=(None, 32, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_2>

### 2.3 Depth 0, Layer 1
For layer 1 of depth 0, the formula for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i}) \times 2$$

Where $i$ is the current depth. 
- Notice that the '$\times~2$' at the end of this expression isn't there for layer 0.


So at depth $i=0$ for layer 1:
$$filters_{0} = 32 \times (2^{0}) \times 2 = 64$$


In [13]:
# creating a Conv3D layer with 64 filters and add relu activation

down_depth_0_layer_1 = Conv3D(filters=64,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                              )(down_depth_0_layer_0)
#building this over down_depth_0_layer_0

#applying activation to it
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1

<KerasTensor shape=(None, 64, 160, 160, 16), dtype=float32, sparse=False, name=keras_tensor_4>

### 2.4 Max pooling
Within the U-Net architecture, there is a max pooling operation after each of the down-convolutions (not including the last down-convolution at the bottom of the U). In general, this means we'll add max pooling after each down-convolution up to (but not including) the `depth - 1` down-convolution (since you started counting at 0). 

For this lab exercise:
- The overall depth of the U-Net we're constructing is 2
- So the bottom of our U is at a depth index of: $2-1 = 1$.
- So far we've only defined the $depth=0$ down-convolutions, so the next thing to do is add max pooling

In [14]:
# defining a max pooling layer
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool

<KerasTensor shape=(None, 64, 80, 80, 8), dtype=float32, sparse=False, name=keras_tensor_5>

#### 2.4.1 Depth 1, Layer 0

At depth 1, layer 0, the formula for calculating the number of filters is:
$$filters_{i} = 32 \times (2^{i})$$

Where $i$ is the current depth.

So at depth $i=1$:
$$filters_{1} = 32 \times (2^{1}) = 64$$



In [15]:
# Add a Conv3D layer to the network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_pool)
# building this on max pool layer
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<KerasTensor shape=(None, 64, 80, 80, 8), dtype=float32, sparse=False, name=keras_tensor_7>

#### 2.4.2 Depth 1,  Layer 1

For layer 1 of depth 1 the formula we'll use for number of filters is:
$$filters_{i} = 32 \times (2^{i}) \times 2$$

Where $i$ is the current depth. 
- Notice that the '$\times 2$' at the end of this expression isn't there for layer 0.

So at depth $i=1$:
$$filters_{0} = 32 \times (2^{1}) \times 2 = 128$$


In [16]:
# Add another Conv3D with 128 filters to the network.
down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<KerasTensor shape=(None, 128, 80, 80, 8), dtype=float32, sparse=False, name=keras_tensor_9>

#### No max pooling at depth 1 (the bottom of the U)

When we get to the "bottom" of the U-net, we don't need to apply max pooling after the convolutions.